### 1. Enable Auto Reloading

In [1]:
%load_ext autoreload
%autoreload 2

### 2. Load Packages & Import External Functions

In [2]:
from json import dump, JSONEncoder
import numpy as np
from copy import deepcopy

from excel_processor import ExcelSet
from dat_processor import DatSet
from utils import *

### 3. Load All Files (Excel & Dat Files)

In [3]:
# read all excels from ./excel_files/
# you can specify a list of fnames as the param to ExcelSet
# e.g. ExcelSet(fnames=['Branch', 'Bus', 'IOCurve', ])
# the default value can be found in excel_processors.py
excel_set = ExcelSet() 

In [4]:
# get all the dat files (shape files & bus load files) from ./dat_files/
# suppose the list of all dat files' is supplied by ShapeFile.xlsx and LoadFile.xlsx
shape_fnames = excel_set.retrieve_col(
    "Shape", ["ShapeFile"]
)
load_fnames = excel_set.retrieve_col(
    "LoadAreaLoad", ["LoadFile"]
)

In [5]:
# read all dat files
shape_set = DatSet([fn[0] for fn in shape_fnames], dir='dat_files')
load_set = DatSet([fn[0] for fn in load_fnames], dir='dat_files')

### 4. Obtain Parameters for UC.jl Input

In [6]:
# get the parameters
params = get_params(excel_set, time_horizon=24*365)

In [7]:
params

{'Version': '0.4',
 'Time horizon (h)': 8760,
 'Power balance penalty ($/MW)': 3500.0,
 'Area-Level Load Shedding Penalty ($/MW; UNUSED)': 4000.0,
 'Load shed panealty ($/MW; UNUSED)': 2000.0}

### 5. Obtain All Generators' Information

In [8]:
# get the mega dictionary for generators
all_units = get_units(excel_set, shape_set)

### 6. Define the Encoder (to Address JSON Output Encoding Issue)

In [9]:
# define the NpEncoder
class NpEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

### 7. Load the Files of Buses

In [10]:
# each area at a given year has an associated .dat file
# the list of dat files will be based on the file LoadAreaLoad.xlsx
# each entry is a tuple of 4 elements: (file_name, area_name, year, area_id)
files = excel_set.retrieve_col("LoadAreaLoad", ["LoadAreaID", "Year", "LoadFile"])
BUS_FILES = []
for file_info in files:
    area_id = file_info[0]
    year = file_info[1]
    fname = file_info[2]
    area_name = excel_set.query_and_retrieve('LoadArea', 'LoadAreaID', [area_id], 'LoadAreaName')
    BUS_FILES.append((fname, area_name, year, area_id, )) # append the tuple to the list

# let us see what the list has
# to simplify, a map function is used to display the fnames only.
for idx, fname in enumerate(BUS_FILES):
    print(idx, fname)

0 ('Load_NORTH_2007.dat', 'NORTH', 2007, 27)
1 ('Load_NORTH_2008.dat', 'NORTH', 2008, 27)
2 ('Load_NORTH_2009.dat', 'NORTH', 2009, 27)
3 ('Load_NORTH_2010.dat', 'NORTH', 2010, 27)
4 ('Load_NORTH_CENTRAL_2007.dat', 'NORTH_CENTRAL', 2007, 28)
5 ('Load_NORTH_CENTRAL_2008.dat', 'NORTH_CENTRAL', 2008, 28)
6 ('Load_NORTH_CENTRAL_2009.dat', 'NORTH_CENTRAL', 2009, 28)
7 ('Load_NORTH_CENTRAL_2010.dat', 'NORTH_CENTRAL', 2010, 28)
8 ('Load_EAST_2007.dat', 'EAST', 2007, 29)
9 ('Load_EAST_2008.dat', 'EAST', 2008, 29)
10 ('Load_EAST_2009.dat', 'EAST', 2009, 29)
11 ('Load_EAST_2010.dat', 'EAST', 2010, 29)
12 ('Load_FAR_WEST_2007.dat', 'FAR_WEST', 2007, 30)
13 ('Load_FAR_WEST_2008.dat', 'FAR_WEST', 2008, 30)
14 ('Load_FAR_WEST_2009.dat', 'FAR_WEST', 2009, 30)
15 ('Load_FAR_WEST_2010.dat', 'FAR_WEST', 2010, 30)
16 ('Load_WEST_2007.dat', 'WEST', 2007, 31)
17 ('Load_WEST_2008.dat', 'WEST', 2008, 31)
18 ('Load_WEST_2009.dat', 'WEST', 2009, 31)
19 ('Load_WEST_2010.dat', 'WEST', 2010, 31)
20 ('Load_WEST_203

In [11]:
# IMPORTANT: Set DEBUG=False to get the complete results
# if DEBUG=True, then we can select a few sample files for testing purpose
DEBUG = True

def filter_with_list_of(attr, file_list, filter_list):
    if attr == 'index':
        return [file_list[i] for i in filter_list]
    if attr == 'area_name':
        return [file_info for file_info in file_list if (file_info[1] in filter_list)]
    if attr == 'year':
        return [file_info for file_info in file_list if (file_info[2] in filter_list)]
    if attr == 'area_id':
        return [file_info for file_info in file_list if (file_info[3] in filter_list)]
    return []
    

if DEBUG:
    # here I select the files at indices 27 & 11 ('Load_COAST_2009.dat', 'Load_EAST_2009.dat')
    # feel free to change your selection, or you can set DEBUG=False to parse all files
    filtered_with_indices = filter_with_list_of('index', BUS_FILES, [7, 11])
    # you can also select by area names, years, area_ids
    # if you want to select just one value, say one year, then just do a list of one number
    # for example, [2010]
    filtered_with_area_names = filter_with_list_of('area_name', BUS_FILES, ['NORTH', 'WEST', 'EAST'])
    filtered_with_years = filter_with_list_of('year', BUS_FILES, [2010, ])
    filtered_with_area_ids = filter_with_list_of('area_id', BUS_FILES, [32, 33])
    
    print("By indices we get: ")
    print(filtered_with_indices)
    print("By area names we get: ")
    print(filtered_with_area_names)
    print("By years we get: ")
    print(filtered_with_years)
    print("By area ids we get: ")
    print(filtered_with_area_ids)
    
    # you can also chain the filtering
    chain_filter = filter_with_list_of('area_name', BUS_FILES, ['EAST', 'WEST', 'COAST'])
    chain_filter = filter_with_list_of('year', chain_filter, [2010, 2009])
    print("By chaining multiple filters we get: ")
    print(chain_filter)
    
    # pick one of the filtered list
    # you can change it to any of the 5 lists defined above
    BUS_FILES = chain_filter

By indices we get: 
[('Load_NORTH_CENTRAL_2010.dat', 'NORTH_CENTRAL', 2010, 28), ('Load_EAST_2010.dat', 'EAST', 2010, 29)]
By area names we get: 
[('Load_NORTH_2007.dat', 'NORTH', 2007, 27), ('Load_NORTH_2008.dat', 'NORTH', 2008, 27), ('Load_NORTH_2009.dat', 'NORTH', 2009, 27), ('Load_NORTH_2010.dat', 'NORTH', 2010, 27), ('Load_EAST_2007.dat', 'EAST', 2007, 29), ('Load_EAST_2008.dat', 'EAST', 2008, 29), ('Load_EAST_2009.dat', 'EAST', 2009, 29), ('Load_EAST_2010.dat', 'EAST', 2010, 29), ('Load_WEST_2007.dat', 'WEST', 2007, 31), ('Load_WEST_2008.dat', 'WEST', 2008, 31), ('Load_WEST_2009.dat', 'WEST', 2009, 31), ('Load_WEST_2010.dat', 'WEST', 2010, 31), ('Load_WEST_2030.dat', 'WEST', 2030, 31)]
By years we get: 
[('Load_NORTH_2010.dat', 'NORTH', 2010, 27), ('Load_NORTH_CENTRAL_2010.dat', 'NORTH_CENTRAL', 2010, 28), ('Load_EAST_2010.dat', 'EAST', 2010, 29), ('Load_FAR_WEST_2010.dat', 'FAR_WEST', 2010, 30), ('Load_WEST_2010.dat', 'WEST', 2010, 31), ('Load_SOUTH_CENTRAL_2010.dat', 'SOUTH_CEN

### 8. Generate JSON Files

In [12]:
# the code in the following coding block will do the following things:
# STEP 1. prepare the mega result dictionary
# - note: each superbus is an area at a given year. 
# - e.g.: NORTH_2007 and NORTH_2009 are two different superbuses.
# STEP 2. enumerate through the BUS_FILES and load the data for each file
# STEP 3. generate a key-val pair for a super-bus using <area_name>_<year>.
# STEP 4. get the base_mw using area_id, and complete the information of the bus load entry.
# STEP 5. obtain the generators using area_name and month, and complete the hourly resource data.

result = {
    'Parameters': params,
    'Buses': {},
    'Generators': {},
    "Transmission lines": {},
    "Interfaces": {},
}

# HAS_INTERFACE=True: uses SUPER bus, transmission flows among super-buses
# HAS_INTERFACE=False: uses MEGA bus, no interface (also no transmission lines)
HAS_INTERFACE = True
# group super bus files by years 
superbus_group_by_year = {bus_file[2]: [] for bus_file in BUS_FILES}

for fname, area_name, year, area_id in BUS_FILES:
    # each dat files contain 365 days, so each file will generator 365 JSON files
    # get the base_mw
    base_mw = float(excel_set.query_and_retrieve(
        'LoadArea', 'LoadAreaID', [area_id], 'BaseLoadMW'
    )) / 1000
    superbus_loads = load_set.get_data_on_date(fname, multiplier=base_mw)
    if superbus_loads is None:
        continue
    superbus_name = f'{area_name}_{year}' # note that now the bus name contains year
    if HAS_INTERFACE:
        result['Buses'].update({
            superbus_name: {
                "Load (MW)": superbus_loads
            },
        })
        superbus_group_by_year[year].append(area_name)
    else:
        megabus_name = f'MEGA_{year}' # each year is a mega bus
        # get the current load and add them
        megabus_info = result['Buses'].get(megabus_name, {})
        megabus_loads = megabus_info.get("Load (MW)", [0.0] * len(superbus_loads))
        megabus_loads = [x + y for x, y in zip(megabus_loads, superbus_loads)]
        result['Buses'].update({
            megabus_name: {
                "Load (MW)": megabus_loads
            },
        })
    # gen units are indep of year numbers
    # we have to change the bus name to include the year number
    all_units_in_area = deepcopy(all_units.get(area_name, {}))
    if HAS_INTERFACE:
        modify_generator_data_with_year(all_units_in_area, year, area_name)
    else:
        modify_generator_data_with_year(all_units_in_area, year)
    result['Generators'].update(all_units_in_area)

In [13]:
# transmission lines & interfaces
if HAS_INTERFACE:
    all_transmission, all_interfaces = get_interface_transmission(excel_set)
    # for the lines, we need to correct the source/target super bus name with year number
    # `superbus_group_by_year` looks like {2010: ["WEST", "EAST"]}
    transmission_temp_dict = {}
    interface_temp_dict = {}
    for year, area_names in superbus_group_by_year.items():
        # update lines
        all_transmission_with_year = deepcopy(all_transmission)
        modify_transmission_data_with_year(all_transmission_with_year, year)
        transmission_temp_dict.update(all_transmission_with_year)
        # update interfaces
        all_interfaces_with_year = deepcopy(all_interfaces)
        modify_interface_data_with_year(all_interfaces_with_year, year)
        interface_temp_dict.update(all_interfaces_with_year)
    
    # now filter all lines 
    all_buses = result['Buses'].keys()
    invalid_lines = []
    print(transmission_temp_dict)
    for line_name, line_dict in transmission_temp_dict.items():
        fbus, tbus = line_dict['Source bus'], line_dict['Target bus']
        valid_src = fbus in all_buses
        valid_targ = tbus in all_buses
        if not (valid_src and valid_targ):
            invalid_lines.append(line_name)
        elif fbus == tbus:
            invalid_lines.append(line_name)
    for key in invalid_lines:
        del transmission_temp_dict[key]
    print(invalid_lines)
    result['Transmission lines'].update(transmission_temp_dict)
    
    # now filter all interfaces
    all_lines = result['Transmission lines'].keys()
    invalid_ifcs = []
    for ifc_name, ifc_dict in interface_temp_dict.items():
        valid_out = set(ifc_dict['Outbound lines']).issubset(all_lines)
        valid_in = set(ifc_dict['Inbound lines']).issubset(all_lines)
        if not (valid_out and valid_in):
            invalid_ifcs.append(ifc_name)
    for key in invalid_ifcs:
        del interface_temp_dict[key]    
    result['Interfaces'].update(interface_temp_dict)

{'l2246_2009': {'Source bus (regular, UNUSED)': 'b40600', 'Target bus (regular, UNUSED)': 'b45972', 'Source bus': 'EAST_2009', 'Target bus': 'COAST_2009', 'Reactance (ohms)': 0.00115999998524785, 'Susceptance (S)': 0.0206599999219179, 'Normal flow limit (MW)': 1710.0, 'Emergency flow limit (MW)': 1710.0}, 'l2604_2009': {'Source bus (regular, UNUSED)': 'b44500', 'Target bus (regular, UNUSED)': 'b44645', 'Source bus': 'COAST_2009', 'Target bus': 'EAST_2009', 'Reactance (ohms)': 0.00346000003628433, 'Susceptance (S)': 0.0336299985647202, 'Normal flow limit (MW)': 1137.0, 'Emergency flow limit (MW)': 1450.0}, 'l2622_2009': {'Source bus (regular, UNUSED)': 'b44645', 'Target bus (regular, UNUSED)': 'b45500', 'Source bus': 'EAST_2009', 'Target bus': 'COAST_2009', 'Reactance (ohms)': 0.00331000005826354, 'Susceptance (S)': 0.033100001513958, 'Normal flow limit (MW)': 1137.0, 'Emergency flow limit (MW)': 1450.0}, 'l2623_2009': {'Source bus (regular, UNUSED)': 'b44645', 'Target bus (regular, UNU

In [14]:
# the code in the following coding block will do the following things:
# STEP 1. prepare the `all_results` which contains data of each group
# - note: the network may not be fully connected, thus, we need to identify 
# - what buses are connected with each other and group them 
# STEP 2. with all buses being grouped, separate the `result` into `sub_result`s
# STEP 3. write to multiple JSON files

all_results = []
if HAS_INTERFACE:
    # we need to detect seperation based on the Transmission Lines
    bus_names = list(result['Buses'].keys())
    line_dict = result['Transmission lines']
    # Let us see what we have at the moment before grouping
    print("We have the following buses: ")
    for i, bus_name in enumerate(bus_names):
        print(bus_name, "with index in matrix =", i)
    print("We have the following lines: ")
    for l, l_info in line_dict.items():
        print(l_info['Source bus'], '->', l_info['Target bus'])
    # now let us group
    groups = group_buses(bus_names, line_dict)
    print(f"We have identified {len(groups)} groups: ")
    print(groups)
    # based on the groups, we need to separate the dictionary
    # which gives us an array of dicts and maybe multiple json files
    for bus_group in groups:
        sub_result = {
            'Parameters': params,
            'Buses': {bus_name: result["Buses"][bus_name] for bus_name in bus_group},
            'Generators': {},
            "Transmission lines": {},
            "Interfaces": {},
        }
        for gen_name, gen_dict in result["Generators"].items():
            if gen_dict["Bus"] in bus_group:
                sub_result["Generators"].update({
                    gen_name: gen_dict
                })
        for line_name, line_dict in result["Transmission lines"].items():
            fbus, tbus = line_dict['Source bus'], line_dict['Target bus']
            valid_src = fbus in bus_group
            valid_targ = tbus in bus_group
            if valid_src and valid_targ:
                sub_result["Transmission lines"].update({
                    line_name: line_dict
                })
        line_group = sub_result['Transmission lines'].keys()
        for ifc_name, ifc_dict in result["Interfaces"].items():
            valid_out = set(ifc_dict['Outbound lines']).issubset(line_group)
            valid_in = set(ifc_dict['Inbound lines']).issubset(line_group)
            if valid_in and valid_out:
                sub_result['Interfaces'].update({
                    ifc_name: ifc_dict
                })
        all_results.append(sub_result)
else:
    all_results = [result]

We have the following buses: 
EAST_2009 with index in matrix = 0
EAST_2010 with index in matrix = 1
WEST_2009 with index in matrix = 2
WEST_2010 with index in matrix = 3
COAST_2009 with index in matrix = 4
COAST_2010 with index in matrix = 5
We have the following lines: 
EAST_2009 -> COAST_2009
COAST_2009 -> EAST_2009
EAST_2009 -> COAST_2009
EAST_2009 -> COAST_2009
EAST_2010 -> COAST_2010
COAST_2010 -> EAST_2010
EAST_2010 -> COAST_2010
EAST_2010 -> COAST_2010
We have identified 4 groups: 
[['EAST_2009', 'COAST_2009'], ['EAST_2010', 'COAST_2010'], ['WEST_2009'], ['WEST_2010']]


In [15]:
# everything is ready, now dump into json file
output_dir = 'uc_jsons' # you can modify this to your preferred directory name
output_fname = 'GridViewSample' # you can modify this to your preferred file name
for i, r in enumerate(all_results): # count from 1
    # to debug, you can re-assign some simplified values to each group result `r`
    """
        for example, I just want to see what my transmission lines & interfaces are, 
        since having all buses and generators would make the JSON file too large and slow.
        Then I can do following:
        r = {
            'Parameters': params,
            'Buses': {}, # the buses' data are manually deleted
            'Generators': {}, # the generators' data are manually deleted
            "Transmission lines": r["Transmission lines"], # the lines' data are kept
            "Interfaces": r["Interfaces"], # the interfaces' data are kept
        }
    """
    uc_jl_fname = f'./{output_dir}/{output_fname}_Group_{i+1}.json'
    with open(uc_jl_fname, 'w', encoding='utf8') as write_file:
        dump(r, write_file, indent=4, cls=NpEncoder)
        print(f"@info: the file has been saved to {uc_jl_fname} (Group: {groups[i]})")

@info: the file has been saved to ./uc_jsons/GridViewSample_Group_1.json (Group: ['EAST_2009', 'COAST_2009'])
@info: the file has been saved to ./uc_jsons/GridViewSample_Group_2.json (Group: ['EAST_2010', 'COAST_2010'])
@info: the file has been saved to ./uc_jsons/GridViewSample_Group_3.json (Group: ['WEST_2009'])
@info: the file has been saved to ./uc_jsons/GridViewSample_Group_4.json (Group: ['WEST_2010'])
